In [2]:
import numpy as np
from PIL import Image
import yaml
from tkinter import Tk, filedialog
import os

def read_pgm(filename):
    """Read a PGM file and return the image data as a numpy array."""
    with open(filename, 'rb') as f:
        header = f.readline()
        assert header == b'P5\n'
        width, height = [int(i) for i in f.readline().split()]
        maxval = int(f.readline())
        image = np.fromfile(f, dtype=np.uint8).reshape((height, width))
    return image, width, height

def write_pgm(filename, image, width, height):
    """Write the numpy array as a PGM file."""
    with open(filename, 'wb') as f:
        f.write(b'P5\n')
        f.write(f'{width} {height}\n'.encode())
        f.write(b'255\n')
        image.tofile(f)

def rotate_map(image, angle):
    """Rotate the map image by a given angle."""
    pil_image = Image.fromarray(image)
    rotated_image = pil_image.rotate(angle, resample=Image.NEAREST, expand=True)
    return np.array(rotated_image)

def find_map_bounds(image):
    """Find the bounds of the actual map in the occupancy grid."""
    rows = np.any(image < 255, axis=1)
    cols = np.any(image < 255, axis=0)
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    return x_min, x_max, y_min, y_max

def adjust_origin(image, x_min, y_min, resolution):
    """Adjust the map origin to the new origin based on the map bounds."""
    adjusted_image = image[y_min:, x_min:]
    new_origin_x = -x_min * resolution
    new_origin_y = -y_min * resolution
    return adjusted_image, new_origin_x, new_origin_y

def update_yaml(yaml_file, new_origin):
    """Update the origin in the YAML file."""
    with open(yaml_file, 'r') as f:
        data = yaml.safe_load(f)
    
    data['origin'] = new_origin

    with open(yaml_file, 'w') as f:
        yaml.safe_dump(data, f, default_flow_style=False)

def main():
    # 파일 선택을 위한 Tkinter 초기화
    root = Tk()
    root.withdraw()  # Tkinter GUI를 표시하지 않음

    # PGM 파일 선택
    map_filename = filedialog.askopenfilename(title="Select PGM File", filetypes=[("PGM files", "*.pgm")])
    if not map_filename:
        print("No PGM file selected. Exiting.")
        return
    
    # YAML 파일 선택
    yaml_filename = filedialog.askopenfilename(title="Select YAML File", filetypes=[("YAML files", "*.yaml")])
    if not yaml_filename:
        print("No YAML file selected. Exiting.")
        return

    # 저장 경로 선택
    save_dir = filedialog.askdirectory(title="Select Directory to Save Adjusted Files")
    if not save_dir:
        print("No directory selected. Exiting.")
        return

    # Load map data
    image, width, height = read_pgm(map_filename)

    # Parameters
    resolution = 0.05  # YAML 파일에서 가져와야 함
    rotation_angle = -5  # 맵을 수평으로 맞추기 위한 회전 각도 (양수면 시계 방향, 음수면 반시계 방향)

    # Rotate the map
    rotated_image = rotate_map(image, rotation_angle)

    # Find the bounds of the actual map
    x_min, x_max, y_min, y_max = find_map_bounds(rotated_image)

    # Adjust the origin to the bottom-left corner of the map
    adjusted_image, new_origin_x, new_origin_y = adjust_origin(rotated_image, x_min, y_min, resolution)

    # Save the adjusted map
    adjusted_map_filename = os.path.join(save_dir, 'adjusted_center.pgm')
    write_pgm(adjusted_map_filename, adjusted_image, adjusted_image.shape[1], adjusted_image.shape[0])

    # Update the YAML file and save to new location
    update_yaml(yaml_filename, [new_origin_x, new_origin_y, 0])

    print(f"Map adjustment complete. Adjusted map saved as '{adjusted_map_filename}' and YAML file updated.")

if __name__ == "__main__":
    main()


TypeError: 'NoneType' object does not support item assignment